<a href="https://colab.research.google.com/github/rrizwan98/SSUET-chatbot/blob/main/SE_chatbot%201.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install tflearn

     |████████████████████████████████| 107 kB 5.3 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=7902cb15b08124299853d200da71038afd5ca30c7504f3caab50e24c294baa0d
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json
with open('/content/drive/MyDrive/intents.json') as file:
    data = json.load(file)


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'Introduction to computing', 'patterns': ['Introduction to computing teacher name?', 'Introduction to computing CREDIT HOURS?', 'itc teacher room?', 'Introduction to computing COURSE CODE'], 'responses': ['itc teacher name: Sir Haris Mehboob', ' ITC CREDIT HOURS:3+1.', 'Introduction to computing Course Code:SWE105', 'itc teacher room:CF:4A'], 'context_set': ''}, {'tag': 'Programing fundamentals', 'patterns': ['what is programming fundamental teacher name?', 'Programing fundamentals CREDIT HOURS?', 'What is the Programing fundamentals teacher room?', 'Programing fundamen

In [5]:
print(data['intents'])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'Introduction to computing', 'patterns': ['Introduction to computing teacher name?', 'Introduction to computing CREDIT HOURS?', 'itc teacher room?', 'Introduction to computing COURSE CODE'], 'responses': ['itc teacher name: Sir Haris Mehboob', ' ITC CREDIT HOURS:3+1.', 'Introduction to computing Course Code:SWE105', 'itc teacher room:CF:4A'], 'context_set': ''}, {'tag': 'Programing fundamentals', 'patterns': ['what is programming fundamental teacher name?', 'Programing fundamentals CREDIT HOURS?', 'What is the Programing fundamentals teacher room?', 'Programing fundamentals COURSE 

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
words = []
labels = []
docs_x = []
docs_y = []

# Tokinization.
for intent in data['intents']:
    for pattern in intent['patterns']:
      wrds = nltk.word_tokenize(pattern)
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [13]:
docs_x

[['Hi'],
 ['How', 'are', 'you'],
 ['Is', 'anyone', 'there', '?'],
 ['Hello'],
 ['Good', 'day'],
 ['Whats', 'up'],
 ['cya'],
 ['See', 'you', 'later'],
 ['Goodbye'],
 ['I', 'am', 'Leaving'],
 ['Have', 'a', 'Good', 'day'],
 ['Introduction', 'to', 'computing', 'teacher', 'name', '?'],
 ['Introduction', 'to', 'computing', 'CREDIT', 'HOURS', '?'],
 ['itc', 'teacher', 'room', '?'],
 ['Introduction', 'to', 'computing', 'COURSE', 'CODE'],
 ['what', 'is', 'programming', 'fundamental', 'teacher', 'name', '?'],
 ['Programing', 'fundamentals', 'CREDIT', 'HOURS', '?'],
 ['What', 'is', 'the', 'Programing', 'fundamentals', 'teacher', 'room', '?'],
 ['Programing', 'fundamentals', 'COURSE', 'CODE'],
 ['what', 'is', 'programming', 'fundamental', 'lab', 'teacher', 'name', '?'],
 ['programming', 'fundamental', 'Lab', 'Room', 'No', '?'],
 ['ssuet', 'Canteens', '?'],
 ['canteens', 'location', '?'],
 ['ssuet', 'canteen', 'timing'],
 ['Library', 'timing'],
 ['what', 'are', 'your', 'hours', 'to', 'open', 'the',

In [14]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [16]:
labels

['Introduction to computing',
 'Library',
 'Programing fundamentals',
 'Programing fundamentals lab',
 'canteen',
 'goodbye',
 'greeting']

In [17]:
# beg of words.
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

[0, 1, 0, 0, 0, 0, 0]

In [ ]:
training = numpy.array(training)
output = numpy.array(output)

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)


Training Step: 4039  | total loss: 0.00853 | time: 0.017s
| Adam | epoch: 1010 | loss: 0.00853 - acc: 1.0000 -- iter: 24/27
Training Step: 4040  | total loss: 0.00892 | time: 0.024s
| Adam | epoch: 1010 | loss: 0.00892 - acc: 1.0000 -- iter: 27/27
--


In [ ]:
model.save("model.tflearn")

INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hello
Hello!
You: canteens location?
canteens location: 2 canteens are in ssuet 1 for boys near parking gate and 2nd for girls opposite of nazami workshop
You: ssuet canteen timing
Welcome to SSUET canteen!
You: Library timing
library timing of SSUET open 9 am 5 pm Monday-Friday! and 1 pm to 2 pm lunch timing.
You: what is programming fundamental lab teacher name?
Programming Fundamental lab teacher Name: Miss Nida khalil.
You: quit
